<a href="https://www.kaggle.com/code/shishirkulal/report-rating?scriptVersionId=286774421" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/asap-2-0/ASAP2_train_sourcetexts.csv


In [4]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset

# 1. Load and Prepare Data
df = pd.read_csv('/kaggle/input/asap-2-0/ASAP2_train_sourcetexts.csv')

# Ensure the columns match your CSV; usually 'score' and 'full_text'
# We normalize scores to a 0-1 range for a regression task
max_val = df['score'].max()
df['label'] = df['score'].astype(float) / max_val

train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)

# 2. Faster Model Choice (DistilRoBERTa)
model_ckpt = "distilbert/distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Reducing max_length to 256 speeds up training significantly
def tokenize_func(batch):
    return tokenizer(batch["full_text"], padding="max_length", truncation=True, max_length=256)

train_ds = Dataset.from_pandas(train_df[['full_text', 'label']]).map(tokenize_func, batched=True)
val_ds = Dataset.from_pandas(val_df[['full_text', 'label']]).map(tokenize_func, batched=True)

# 3. Load Model and Freeze early layers to save time
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=1)

# Freeze all layers except the last 2 and the classifier head
for param in model.roberta.embeddings.parameters():
    param.requires_grad = False
for layer in model.roberta.encoder.layer[:-2]:
    for param in layer.parameters():
        param.requires_grad = False

# 4. Optimized Training Arguments
training_args = TrainingArguments(
    output_dir="./fast_report_rater",
    eval_strategy="epoch",
    save_strategy="no",             # Don't save checkpoints during training to save disk speed
    learning_rate=5e-5,
    per_device_train_batch_size=16, # Increased batch size for speed
    num_train_epochs=2,             # 2 epochs is usually enough for a rater
    weight_decay=0.01,
    fp16=True,                      # Use GPU acceleration (Mixed Precision)
    report_to="none",
    logging_steps=50
)

# 5. Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
)

# 6. Execute Training
print("Starting fast training...")
trainer.train()

# 7. Save for later use
model.save_pretrained("./final_rater_model")
tokenizer.save_pretrained("./final_rater_model")
print("Model saved to './final_rater_model'")

# --- QUICK PREDICTION FUNCTION ---

def get_report_rating(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Scale back to original score range
    prediction = torch.sigmoid(outputs.logits).item()
    return round(prediction * max_val, 2)

# Test it
# print(f"Sample Rating: {get_report_rating('The project analyzes the impact of renewable energy...')}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting fast training...


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.018900,0.016157
2,0.017400,0.016629


Model saved to './final_rater_model'


In [7]:
!pip install textstat language_tool_python

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.6/176.6 kB 4.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.3 MB/s eta 0:00:00a 0:00:01


In [8]:
import torch
import textstat
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. Load the model you just trained
model_path = "./final_rater_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval() # Set to evaluation mode

def analyze_report(text):
    # --- Part A: Scoring (The AI Model) ---
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Scale score to 0-100 (adjust max_val if your dataset max was different)
    max_val = 10 
    raw_prediction = torch.sigmoid(outputs.logits).item()
    ai_score = round(raw_prediction * 100, 1)

    # --- Part B: Generating Suggestions (Rule-based) ---
    suggestions = []
    
    # Check 1: Length
    word_count = len(text.split())
    if word_count < 300:
        suggestions.append("📍 **Increase Depth:** Your report is quite short. High-scoring reports usually provide more detailed evidence and analysis.")
    
    # Check 2: Readability
    read_score = textstat.flesch_reading_ease(text)
    if read_score < 40:
        suggestions.append("📍 **Simplify Language:** Your 'Reading Ease' score is low. Try breaking down long, complex sentences to make your points clearer.")
    elif read_score > 80:
        suggestions.append("📍 **Professional Tone:** The language is very simple. Consider using more technical or academic vocabulary relevant to the project.")

    # Check 3: Structural Keywords
    keywords = ["conclusion", "references", "methodology", "introduction"]
    missing = [k for k in keywords if k not in text.lower()]
    if missing:
        suggestions.append(f"📍 **Improve Structure:** Your report seems to be missing clear sections for: {', '.join(missing)}.")

    return {
        "Score": ai_score,
        "Suggestions": suggestions if suggestions else ["Perfect! Your report meets all structural and readability standards."]
    }

# --- TEST IT ---
sample_text = "Your project report text goes here..."
results = analyze_report(sample_text)

print(f"--- REPORT CARD ---")
print(f"Final Grade: {results['Score']}/100")
print("\nHow to improve:")
for s in results['Suggestions']:
    print(s)

--- REPORT CARD ---
Final Grade: 60.6/100

How to improve:
📍 **Increase Depth:** Your report is quite short. High-scoring reports usually provide more detailed evidence and analysis.
📍 **Professional Tone:** The language is very simple. Consider using more technical or academic vocabulary relevant to the project.
📍 **Improve Structure:** Your report seems to be missing clear sections for: conclusion, references, methodology, introduction.


In [10]:
import torch
import textstat
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. Load the model and tokenizer from your saved path
MODEL_PATH = "./final_rater_model"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
model.eval()

# 2. Define the Prediction & Logic Function
def analyze_report(text):
    if len(text.strip()) < 20:
        return "Please enter a valid report (too short)."

    # --- AI Scoring ---
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Assuming training normalized scores to 0-1, we scale to 100
    raw_prediction = torch.sigmoid(outputs.logits).item()
    ai_score = round(raw_prediction * 100, 1)

    # --- Suggestion Engine ---
    suggestions = []
    
    # Word Count Check
    words = text.split()
    if len(words) < 250:
        suggestions.append("❌ <b>Depth:</b> Your report is very short. Expand your arguments with data or examples.")
    
    # Readability Check
    read_score = textstat.flesch_reading_ease(text)
    if read_score < 45:
        suggestions.append("❌ <b>Readability:</b> Sentences are too complex. Try using shorter sentences to explain your points.")
    
    # Structure Check
    structure_points = ["introduction", "conclusion", "method", "result"]
    missing = [p for p in structure_points if p not in text.lower()]
    if missing:
        suggestions.append(f"❌ <b>Structure:</b> You might be missing these key sections: {', '.join(missing)}.")

    # Format the Output
    suggestion_html = "".join([f"<li>{s}</li>" for s in suggestions])
    if not suggestions:
        suggestion_html = "<li>✅ Great job! Your report is well-structured and readable.</li>"

    return ai_score, suggestion_html

# 3. Create UI Elements
text_area = widgets.Textarea(
    placeholder='Paste your project report here...',
    description='Report:',
    layout={'height': '300px', 'width': '100%'}
)

button = widgets.Button(
    description='Rate My Report',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    layout={'width': 'max-content'}
)

output = widgets.Output()

# 4. Button Click Event
def on_button_clicked(b):
    with output:
        clear_output()
        report_content = text_area.value
        if not report_content:
            print("Please paste a report first!")
            return
            
        print("Analyzing... please wait.")
        score, feedback = analyze_report(report_content)
        
        # Display Results in HTML for a nice look
        display(HTML(f"""
            <div style="border: 2px solid #4CAF50; padding: 20px; border-radius: 10px; background-color: #blue;">
                <h2 style="color: #4CAF50; margin-top: 0;">Analysis Result</h2>
                <div style="font-size: 24px; font-weight: bold;">
                    Overall Score: <span style="color: #2196F3;">{score}/100</span>
                </div>
                <hr>
                <h4 style="margin-bottom: 5px;">Suggestions for Improvement:</h4>
                <ul style="line-height: 1.6;">
                    {feedback}
                </ul>
            </div>
        """))

button.on_click(on_button_clicked)

# 5. Display the UI
display(HTML("<h3>🚀 Project Report Rater (AI Powered)</h3>"))
display(text_area, button, output)

Textarea(value='', description='Report:', layout=Layout(height='300px', width='100%'), placeholder='Paste your…

Button(button_style='success', description='Rate My Report', layout=Layout(width='max-content'), style=ButtonS…

Output()

In [11]:
!pip install textstat docx2txt PyPDF2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00a 0:00:01


In [12]:
import io
import docx2txt
import PyPDF2
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# --- File Extraction Helper ---
def extract_text_from_upload(uploaded_file):
    content = uploaded_file['content']
    filename = uploaded_file['name']
    
    if filename.endswith('.txt'):
        return content.decode('utf-8')
    
    elif filename.endswith('.docx'):
        # Use BytesIO to treat the uploaded bytes as a file
        return docx2txt.process(io.BytesIO(content))
    
    elif filename.endswith('.pdf'):
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(content))
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text
    return None

# --- UI Setup ---
uploader = widgets.FileUpload(
    accept='.txt,.pdf,.docx',  # Only allow these types
    multiple=False 
)

output_area = widgets.Output()

def on_upload_change(change):
    with output_area:
        clear_output()
        if not uploader.value:
            return
        
        # Get the first file from the upload list
        uploaded_file = uploader.value[0]
        print(f"Reading: {uploaded_file['name']}...")
        
        text = extract_text_from_upload(uploaded_file)
        
        if text:
            # Call your analyze_report function from the previous step
            score, feedback = analyze_report(text)
            
            display(HTML(f"""
                <div style="border: 2px solid #2196F3; padding: 15px; border-radius: 10px;">
                    <h3 style="color: #2196F3;">Analysis for: {uploaded_file['name']}</h3>
                    <p style="font-size: 20px;"><b>AI Score: {score}/100</b></p>
                    <ul>{feedback}</ul>
                </div>
            """))
        else:
            print("Unsupported file format.")

uploader.observe(on_upload_change, names='value')

display(HTML("<h4>📁 Upload your Project Report (.txt, .pdf, .docx)</h4>"))
display(uploader, output_area)

FileUpload(value=(), accept='.txt,.pdf,.docx', description='Upload')

Output()